# Importing and reusing `pidgy` literature

A constraint consistent across most programming languages is that
programs are executed line-by-line without any
statements or expressions. raising exceptions 
If literate programs have the computational quality that they __restart
and run all__ the they should 
When `pidgy` programs have this quality they can <code>import</code> in [Python], they become importable essays or reports.

<!--

In [1]:
    __all__ = 'pidgyLoader',
    import pidgy, sys, IPython, mistune as markdown, importnb, IPython as python
    with importnb.Notebook(lazy=True):
        try: from . import tangle, extras
        except: import tangle, extras
    if __name__ == '__main__':
        shell = get_ipython()

    __all__ = 'pidgyLoader',
    import pidgy, sys, IPython, mistune as markdown, importnb, IPython as python
    with importnb.Notebook(lazy=True):
        try: from . import tangle, extras
        except: import tangle, extras
    if __name__ == '__main__':
        shell = get_ipython()

-->

The `pidgyLoader` customizes [Python]'s ability to discover 
[Markdown] and `pidgy` [Notebook]s have the composite `".md.ipynb"` extension.
`importnb` provides a high level API for modifying how content
[Python] imports different file types.

`sys.meta_path and sys.path_hooks`

In [9]:
    class pidgyLoader(importnb.Notebook): 
        extensions = ".md .md.ipynb".split()

    class pidgyLoader(importnb.Notebook): 
        extensions = ".md .md.ipynb".split()

`get_data` determines how a file is decoding from disk.  We use it to make an escape hatch for markdown files otherwise we are importing a notebook.

In [10]:
    def get_data(self, path):
        if self.path.endswith('.md'):
            return self.code(self.decode())
        return super(pidgyLoader, self).get_data(path)

    def get_data(self, path):
        if self.path.endswith('.md'):
            return self.code(self.decode())
        return super(pidgyLoader, self).get_data(path)

The `code` method tangles the [Markdown] to [Python] before compiling to an [Abstract Syntax Tree].

In [11]:
    def code(self, str): 
        return ''.join(extras.demojize(''.join(tangle.pidgy.transform_cell(str))))

    def code(self, str): 
        with importnb.Notebook(lazy=True):
            try: from . import tangle
            except: import tangle
        return ''.join(tangle.pidgy.transform_cell(str))

The `visit` method allows custom [Abstract Syntax Tree] transformations to be applied.

In [5]:
    def visit(self, node):
        return extras.ExtraSyntax().visit(node)

        def visit(self, node):
            with importnb.Notebook():
                try: from . import tangle
                except: import tangle
            return tangle.ReturnYield().visit(node)
        

Attach these methods to the `pidgy` loader.

In [6]:
    pidgyLoader.code, pidgyLoader.visit = code, visit
    pidgyLoader.get_source = pidgyLoader.get_data = get_data

    pidgyLoader.code, pidgyLoader.visit = code, visit
    pidgyLoader.get_source = pidgyLoader.get_data = get_data

The `pidgy` `loader` configures how [Python] discovers modules when they are
imported.
Usually the loader is used as a content manager and in this case we hold the enter 
the context, but do not leave it until `unload_ipython_extension` is executed.

In [7]:
    def load_ipython_extension(shell):
        setattr(shell, 'loaders', getattr(shell, 'loaders', {}))
        shell.loaders[pidgyLoader] = pidgyLoader(position=-1)
        shell.loaders[pidgyLoader].__enter__()

    def load_ipython_extension(shell):
        setattr(shell, 'loaders', getattr(shell, 'loaders', {}))
        shell.loaders[pidgyLoader] = pidgyLoader(position=-1, lazy=True)
        shell.loaders[pidgyLoader].__enter__()

<!--

In [8]:
    
    def unload_ipython_extension(shell): 
        loader = shell.loaders.pop(pidgyLoader)
        loader and loader.__exit__(None, None, None)

-->